### Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from EricWang.utils_ew import *

### 1. Key、Y、Test_FLG

In [3]:
!head -2 data/training-set.csv

Policy_Number,Next_Premium
8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414


In [4]:
# 讀檔
trn_df = pd.read_csv('data/training-set.csv')
test_df = pd.read_csv('data/testing-set.csv')

In [5]:
display_all(trn_df.head())

,Policy_Number,Next_Premium
0,8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414
1,3f9f289dd1b20494a2fe4b1f0fe2d78333c97eda,8236
2,9864c955e499d1fa908f5dc7292054a232f01722,6153
3,92170afa649706baf10774dc0a46e67617c0d50d,12135
4,d16c11ef69f5f6bac02ed18fb1e372394563697f,4345


In [6]:
# 設定Flg
trn_df['Test_Flg'] = '0'
test_df['Test_Flg'] = '1'

all_id_y = pd.concat([trn_df, test_df]).reset_index(drop=True)
all_id_y.head(2)

,Policy_Number,Next_Premium,Test_Flg
0,8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414,0
1,3f9f289dd1b20494a2fe4b1f0fe2d78333c97eda,8236,0


In [7]:
all_id_y = pd.read_feather('data/mdata/all_id_y')

In [8]:
# 資料匯出，檔案包含Key、Y、Test_FLG
all_id_y.to_feather('data/mdata/all_id_y')
all_id_y.head(2)

,Policy_Number,Next_Premium,Test_Flg
0,8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414,0
1,3f9f289dd1b20494a2fe4b1f0fe2d78333c97eda,8236,0


### 2. 整理保單相關Features

In [9]:
# 讀取屬性檔RawData
policy_df = pd.read_csv('data/policy_0702.csv')
claim_df = pd.read_csv('data/claim_0702.csv')

In [10]:
id_keep = ['Policy_Number', 'Prior_Policy_Number', 'Cancellation']

# 保單相關欄位可分成distinct欄位跟dup欄位
policy_rel_dupl = ['Main_Insurance_Coverage_Group',
                   'Insurance_Coverage',
                   'Insured_Amount1',
                   'Insured_Amount2',
                   'Insured_Amount3',
                   'Coverage_Deductible_if_applied',
                   'Premium']
policy_rel_keep = ['Replacement_cost_of_insured_vehicle',
                   'Distribution_Channel',
                   'Multiple_Products_with_TmNewa_(Yes_or_No?)',
                   'lia_class',
                   'plia_acc',
                   'pdmg_acc'
                  ]
keep = id_keep + policy_rel_dupl + policy_rel_keep

#### 保單資料中無重複性之features

In [11]:
policy_part1 = policy_df[id_keep + policy_rel_keep]

# 去重複+重設index
policy_part1 = policy_part1.drop_duplicates()
policy_part1.reset_index(drop=True, inplace=True)

# check
print(len(policy_part1))
display_all(policy_part1.head(2))

351273


,Policy_Number,Prior_Policy_Number,Cancellation,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc
0,79110176bf64b5094c19aad785aeac56e36cb609,e1379d9c360fe4a20027b8acc161ec93f2727e66,,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0
1,61a449f6180b23379a584af904d7412205d8ff8c,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0


In [12]:
# check
# Prior_Policy_Number是用來串資料的key可以拿掉，如果只是要表達有沒有前手保單，用Cancellation即可
policy_part1[policy_part1.Cancellation == 'Y'].Prior_Policy_Number.value_counts()

Series([], Name: Prior_Policy_Number, dtype: int64)

In [13]:
policy_part1.drop(columns=['Prior_Policy_Number'], inplace=True)

In [26]:
# # 對主通路做處理，觀察後取前7大通路，剩下的通路整併成1個
# # policy_part1.Distribution_Channel.value_counts()
# top7 = policy_part1.Distribution_Channel.value_counts()[:7]
# top7

# # change pd values from list
# policy_part1.Distribution_Channel = policy_part1.Distribution_Channel.apply(
#     lambda x: x if x in top7 else 'other_channels')

# policy_part1.head(3)

39005d92752d56310b3a357f42b57f304621689a    176354
5037717f120cf25bbeea6a42e62676bc5c911b12     38128
315b39a0fc19fb45d146c17b813d46c7acb0b872     36493
cc7533150432c2d0fe3dbb2b31308d4774bf63e0     12145
b5b69d907b43e39e97633b946d6c6d0111d94b75      6319
a092b2d4c9d52807932bfedae96b01210db7e06b      5259
4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e      5227
Name: Distribution_Channel, dtype: int64

In [66]:
policy_part1.dtypes

Policy_Number                                  object
Cancellation                                   object
Replacement_cost_of_insured_vehicle           float64
Distribution_Channel                           object
Multiple_Products_with_TmNewa_(Yes_or_No?)    float64
lia_class                                       int64
plia_acc                                      float64
pdmg_acc                                      float64
dtype: object

In [18]:
display_all(policy_part1.head(3))

,Policy_Number,Cancellation,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc
0,79110176bf64b5094c19aad785aeac56e36cb609,,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0
1,61a449f6180b23379a584af904d7412205d8ff8c,,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0
2,1a1f5f582bf1ff23467352e8f2fd31acefddc654,,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0


In [14]:
# Finish!
# policy_part1.to_feather('data/mdata/policy_part1')

# 未處理類別變數
policy_part1.to_feather('data/mdata/policy_part1_v2')

#### 保單資料中具重複性之features

In [15]:
policy_part2 = policy_df[id_keep + policy_rel_dupl]

##### 1.從自負額開始處理

In [16]:
# 特定險種代號的自負額欄位值與自負額無關，改設為0
Insurance_Coverage_no_cov = ['09I', '10A', '14E', '15F', 
                             '15O', '20B', '20K', '29K', 
                             '32N', '33F', '33O', '56K', 
                             '65K']
flt = policy_part2['Insurance_Coverage'].isin(Insurance_Coverage_no_cov)
idx = policy_part2[flt].index

# It's ok
policy_part2.loc[idx, 'Coverage_Deductible_if_applied'] = 0

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
# Look：退保的編號到17XXXX，Train Set的編號到210763
flt = policy_part2.Coverage_Deductible_if_applied < 0
# policy_part2[flt]

看來退保的資料都只存在於Training Set，對此先設一個刪除的flg

In [19]:
# It's ok
policy_part2['policy_cancel_flg'] = np.where(flt, 1, 0)

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
policy_part2_flt = policy_part2.copy()

In [21]:
# Look，退保不能砍，會少15筆rows(Train/Test都有少)
# policy_part2_flt.Policy_Number.drop_duplicates().count()

# 將自負額代碼轉換成金額，代號取平均，比例取保額*比例
# policy_part2_flt.Coverage_Deductible_if_applied.value_counts()

In [22]:
# Warning is ok
sum_insured = (policy_part2_flt[flt].Insured_Amount1 + policy_part2_flt[flt].Insured_Amount2 + policy_part2_flt[flt].Insured_Amount3)

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == 10)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.1

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == 20)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.2


flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == 1)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = 5000

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == 2)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = 6500

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == 3)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = 7666

In [23]:
# 退保的部分也要做轉換
flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == -10)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.1

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == -20)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.2


flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == -1)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = -5000

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == -2)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = -6500

flt = (policy_part2_flt['Coverage_Deductible_if_applied'] == -3)
policy_part2_flt.loc[flt, 'Coverage_Deductible_if_applied'] = -7666

##### 2.再來處理sum_of_premium

In [24]:
# 做出一欄新的Premium；哪個有用不確定，所以都先留著
flt = (policy_part2_flt.policy_cancel_flg == 1)

policy_part2_flt['Premium_except_cancel'] = policy_part2_flt.Premium
policy_part2_flt.loc[flt, 'Premium_except_cancel'] = 0

In [25]:
# 處理過程動到欄位，補missing值
policy_part2_flt.Coverage_Deductible_if_applied.fillna(0, inplace=True)

In [26]:
policy_part2_flt.head(2)

,Policy_Number,Prior_Policy_Number,Cancellation,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,policy_cancel_flg,Premium_except_cancel
0,79110176bf64b5094c19aad785aeac56e36cb609,e1379d9c360fe4a20027b8acc161ec93f2727e66,,車責,16G,2000000,0,10000000,0.0,1551,0,1551
1,79110176bf64b5094c19aad785aeac56e36cb609,e1379d9c360fe4a20027b8acc161ec93f2727e66,,車責,16P,0,0,300000,0.0,1163,0,1163


有三層Level要處理；sum by policy number, sum by coverage group, sum by insurance coverage

In [27]:
drop_column = ['Prior_Policy_Number', 'Cancellation']
group1 = ['Main_Insurance_Coverage_Group']
group2 = ['Insurance_Coverage']
use_in_group2_only = ['policy_cancel_flg']

###### 2-1.sum_group1

In [28]:
tmp_drop_col = drop_column + group1 + group2 + use_in_group2_only

In [29]:
# drop columns
sum_group1 = policy_part2_flt.drop(columns=tmp_drop_col)
# sum
# sum_group1 = sum_group1.groupby('Policy_Number', as_index=False).sum()
sum_group1 = sum_group1.groupby('Policy_Number', as_index=False).agg(['min', 'max', 'sum', 'mean'])

In [30]:
sum_group1.head()

Insured_Amount1                    \
                                                     min      max      sum   
Policy_Number                                                                
000061141b237e8619efedcd6939fddeff05b9a5               0   500000   500000   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5               0  2000000  2200000   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b               0  3000000  3000000   
000148bbf40f28bfc860c0d272aadc429e19e2d1               0  1000000  1000000   
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c               0  1000000  1010000   

                                                         Insured_Amount2  \
                                                    mean             min   
Policy_Number                                                              
000061141b237e8619efedcd6939fddeff05b9a5   166666.666667               0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5   314285.714286               0   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b  1000000.000000               0   
000148bbf40f28bfc860c0d272aadc429e19e2d1   500000.000000               0   
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c   336666.666667               0   

                                                                           \
                                              max      sum           mean   
Policy_Number                                                               
000061141b237e8619efedcd6939fddeff05b9a5        0        0       0.000000   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  2000000  2000000  285714.285714   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b        0        0       0.000000   
000148bbf40f28bfc860c0d272aadc429e19e2d1        0        0       0.000000   
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c        0        0       0.000000   

                                         Insured_Amount3            \
                                                     min       max   
Policy_Number                                                        
000061141b237e8619efedcd6939fddeff05b9a5          100000   1000000   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5           30000  24000000   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b          300000   6000000   
000148bbf40f28bfc860c0d272aadc429e19e2d1          200000  12000000   
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c           50000   2000000   

                                             ...       \
                                             ...        
Policy_Number                                ...        
000061141b237e8619efedcd6939fddeff05b9a5     ...        
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5     ...        
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b     ...        
000148bbf40f28bfc860c0d272aadc429e19e2d1     ...        
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c     ...        

                                         Coverage_Deductible_if_applied       \
                                                                    sum mean   
Policy_Number                                                                  
000061141b237e8619efedcd6939fddeff05b9a5                            0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5                            0.0  0.0   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b                            0.0  0.0   
000148bbf40f28bfc860c0d272aadc429e19e2d1                            0.0  0.0   
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c                            0.0  0.0   

                                         Premium                           \
                                             min   max   sum         mean   
Policy_Number                                                               
000061141b237e8619efedcd6939fddeff05b9a5     199   727  1586   528.666667   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5      80  5352  9132  1304.571429   
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b     780  1280  2861   953.666667   
000148bbf40f28bfc860c0d272aadc429e19e2d1     837   891  1728   864.00

In [31]:
new_col_NM = []
for item in sum_group1.columns.values:
    item = item[::-1]
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[1])
    else:
        new_col_NM.append('_'.join(item))
#         print('_'.join(item))

In [32]:
sum_group1.columns = new_col_NM
sum_group1.reset_index(inplace=True)
display_all(sum_group1.head(2))

,Policy_Number,min_Insured_Amount1,max_Insured_Amount1,sum_Insured_Amount1,mean_Insured_Amount1,min_Insured_Amount2,max_Insured_Amount2,sum_Insured_Amount2,mean_Insured_Amount2,min_Insured_Amount3,max_Insured_Amount3,sum_Insured_Amount3,mean_Insured_Amount3,min_Coverage_Deductible_if_applied,max_Coverage_Deductible_if_applied,sum_Coverage_Deductible_if_applied,mean_Coverage_Deductible_if_applied,min_Premium,max_Premium,sum_Premium,mean_Premium,min_Premium_except_cancel,max_Premium_except_cancel,sum_Premium_except_cancel,mean_Premium_except_cancel
0,000061141b237e8619efedcd6939fddeff05b9a5,0,500000,500000,166666.666667,0,0,0,0.000000,100000,1000000,2100000,7.000000e+05,0.0,0.0,0.0,0.0,199,727,1586,528.666667,199,727,1586,528.666667
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,0,2000000,2200000,314285.714286,0,2000000,2000000,285714.285714,30000,24000000,27546000,3.935143e+06,0.0,0.0,0.0,0.0,80,5352,9132,1304.571429,80,5352,9132,1304.571429


In [33]:
sum_group1.dtypes

Policy_Number                           object
min_Insured_Amount1                      int64
max_Insured_Amount1                      int64
sum_Insured_Amount1                      int64
mean_Insured_Amount1                   float64
min_Insured_Amount2                      int64
max_Insured_Amount2                      int64
sum_Insured_Amount2                      int64
mean_Insured_Amount2                   float64
min_Insured_Amount3                      int64
max_Insured_Amount3                      int64
sum_Insured_Amount3                      int64
mean_Insured_Amount3                   float64
min_Coverage_Deductible_if_applied     float64
max_Coverage_Deductible_if_applied     float64
sum_Coverage_Deductible_if_applied     float64
mean_Coverage_Deductible_if_applied    float64
min_Premium                              int64
max_Premium                              int64
sum_Premium                              int64
mean_Premium                           float64
min_Premium_e

In [34]:
# sum_group1.to_feather('data/mdata/policy_sum_group1')
sum_group1.to_feather('data/mdata/policy_sum_group1_v2')

###### 2-2.sum_group2

In [35]:
# 改保留group1
tmp_drop_col = drop_column + group2 + use_in_group2_only

In [36]:
# drop columns
sum_group2 = policy_part2_flt.drop(columns=tmp_drop_col)

# sum
sum_group2 = sum_group2.groupby(['Policy_Number', 'Main_Insurance_Coverage_Group'], 
                                as_index=False).agg(['min', 'max', 'sum', 'mean'])
sum_group2.head(4)

Insured_Amount1  \
                                                                                   min   
Policy_Number                            Main_Insurance_Coverage_Group                   
000061141b237e8619efedcd6939fddeff05b9a5 車責                                          0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                          0   
                                         車損                                          0   
                                         車責                                          0   

                                                                                 \
                                                                            max   
Policy_Number                            Main_Insurance_Coverage_Group            
000061141b237e8619efedcd6939fddeff05b9a5 車責                              500000   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                   0   
                                         車損                                   0   
                                         車責                             2000000   

                                                                                 \
                                                                            sum   
Policy_Number                            Main_Insurance_Coverage_Group            
000061141b237e8619efedcd6939fddeff05b9a5 車責                              500000   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                   0   
                                         車損                                   0   
                                         車責                             2200000   

                                                                                       \
                                                                                 mean   
Policy_Number                            Main_Insurance_Coverage_Group                  
000061141b237e8619efedcd6939fddeff05b9a5 車責                             166666.666667   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                  0.000000   
                                         車損                                  0.000000   
                                         車責                             733333.333333   

                                                                       Insured_Amount2  \
                                                                                   min   
Policy_Number                            Main_Insurance_Coverage_Group                   
000061141b237e8619efedcd6939fddeff05b9a5 車責                                          0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                          0   
                                         車損                                          0   
                                         車責                                          0   

                                                                                 \
                                                                            max   
Policy_Number                            Main_Insurance_Coverage_Group            
000061141b237e8619efedcd6939fddeff05b9a5 車責                                   0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                   0   
                                         車損                                   0   
                                         車責                             2000000   

                                                                                 \
                                                                            sum   
Policy_Number                            Main_Insurance_Coverage_Group            
000061141b237e8619efedcd6939fddeff05b9a5 車責                                   0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5 竊盜                                   0   
                        

In [37]:
# pivot
sum_group2= sum_group2.pivot_table(index='Policy_Number', 
                                   columns='Main_Insurance_Coverage_Group').fillna(0)

# 重設index，把policy_number拋出來
sum_group2.reset_index(inplace = True)

sum_group2.head()

Policy_Number  \
                                                                          
Main_Insurance_Coverage_Group                                             
0                              000061141b237e8619efedcd6939fddeff05b9a5   
1                              0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5   
2                              0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b   
3                              000148bbf40f28bfc860c0d272aadc429e19e2d1   
4                              0001ad0ad0618f6d2e2c1875105adce6bcd77c2c   

                              Coverage_Deductible_if_applied                 \
                                                         max           mean   
Main_Insurance_Coverage_Group                             竊盜   車損   車責   竊盜   
0                                                        0.0  0.0  0.0  0.0   
1                                                        0.0  0.0  0.0  0.0   
2                                                        0.0  0.0  0.0  0.0   
3                                                        0.0  0.0  0.0  0.0   
4                                                        0.0  0.0  0.0  0.0   

                                                         ...    \
                                         min             ...     
Main_Insurance_Coverage_Group   車損   車責   竊盜   車損   車責   ...     
0                              0.0  0.0  0.0  0.0  0.0   ...     
1                              0.0  0.0  0.0  0.0  0.0   ...     
2                              0.0  0.0  0.0  0.0  0.0   ...     
3                              0.0  0.0  0.0  0.0  0.0   ...     
4                              0.0  0.0  0.0  0.0  0.0   ...     

                              Premium_except_cancel                 \
                                                max   mean           
Main_Insurance_Coverage_Group                    車責     竊盜      車損   
0                                             727.0    0.0     0.0   
1                                            1372.0  430.5  2716.0   
2                                            1280.0    0.0     0.0   
3                                             891.0    0.0     0.0   
4                                            1024.0    0.0   286.0   

                                                                               \
                                             min                  sum           
Main_Insurance_Coverage_Group          車責     竊盜     車損     車責     竊盜      車損   
0                              528.666667    0.0    0.0  199.0    0.0     0.0   
1                              946.333333  200.0   80.0  223.0  861.0  5432.0   
2                              953.666667    0.0    0.0  780.0    0.0     0.0   
3                              864.000000    0.0    0.0  837.0    0.0     0.0   
4                              954.500000    0.0  286.0  885.0    0.0   286.0   

                                       
                                       
Main_Insurance_Coverage_Group      車責  
0                              1586.0  
1                              2839.0  
2                              2861.0  
3                              1728.0  
4                              1909.0  

[5 rows x 73 columns]

In [38]:
sum_group2.columns

MultiIndex(levels=[['Coverage_Deductible_if_applied', 'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3', 'Premium', 'Premium_except_cancel', 'Policy_Number'], ['max', 'mean', 'min', 'sum', ''], ['竊盜', '車損', '車責', '']],
           labels=[[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], [4, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [3, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[None, None, 'Main_Insurance_Coverage_Group'])

In [39]:
new_col_NM = []
for item in sum_group2.columns.values:
    item = item[::-1] #保留except_cancel，後面資料比較好處理
    if item[0] == '':
        new_col_NM.append(item[2])
#         print(item[2])
    else:
        new_col_NM.append('_'.join(item))
#         print('_'.join(item))

In [40]:
sum_group2.columns = new_col_NM
sum_group2.head(2)

,Policy_Number,竊盜_max_Coverage_Deductible_if_applied,車損_max_Coverage_Deductible_if_applied,車責_max_Coverage_Deductible_if_applied,竊盜_mean_Coverage_Deductible_if_applied,車損_mean_Coverage_Deductible_if_applied,車責_mean_Coverage_Deductible_if_applied,竊盜_min_Coverage_Deductible_if_applied,車損_min_Coverage_Deductible_if_applied,車責_min_Coverage_Deductible_if_applied,...,車責_max_Premium_except_cancel,竊盜_mean_Premium_except_cancel,車損_mean_Premium_except_cancel,車責_mean_Premium_except_cancel,竊盜_min_Premium_except_cancel,車損_min_Premium_except_cancel,車責_min_Premium_except_cancel,竊盜_sum_Premium_except_cancel,車損_sum_Premium_except_cancel,車責_sum_Premium_except_cancel
0,000061141b237e8619efedcd6939fddeff05b9a5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,727.0,0.0,0.0,528.666667,0.0,0.0,199.0,0.0,0.0,1586.0
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1372.0,430.5,2716.0,946.333333,200.0,80.0,223.0,861.0,5432.0,2839.0


In [41]:
filter_df = sum_group2.describe(percentiles=[0.9, 0.95, 0.99]).T

In [42]:
filter_df.head(2)

,count,mean,std,min,50%,90%,95%,99%,max
竊盜_max_Coverage_Deductible_if_applied,351273.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
車損_max_Coverage_Deductible_if_applied,351273.0,201.997187,1446.651544,-100000.0,0.0,0.0,0.0,5000.0,100000.0


In [43]:
flt = (filter_df['90%'] == 0)
sum(flt)

26

In [44]:
drop_NM = filter_df[flt].index.values
sum_group2_v2 = sum_group2.drop(columns=drop_NM)
sum_group2_v2.head(2)

,Policy_Number,車損_max_Insured_Amount1,車責_max_Insured_Amount1,車損_mean_Insured_Amount1,車責_mean_Insured_Amount1,車損_min_Insured_Amount1,車損_sum_Insured_Amount1,車責_sum_Insured_Amount1,車責_max_Insured_Amount2,車責_mean_Insured_Amount2,...,車責_max_Premium_except_cancel,竊盜_mean_Premium_except_cancel,車損_mean_Premium_except_cancel,車責_mean_Premium_except_cancel,竊盜_min_Premium_except_cancel,車損_min_Premium_except_cancel,車責_min_Premium_except_cancel,竊盜_sum_Premium_except_cancel,車損_sum_Premium_except_cancel,車責_sum_Premium_except_cancel
0,000061141b237e8619efedcd6939fddeff05b9a5,0.0,500000.0,0.0,166666.666667,0.0,0.0,500000.0,0.0,0.000000,...,727.0,0.0,0.0,528.666667,0.0,0.0,199.0,0.0,0.0,1586.0
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,0.0,2000000.0,0.0,733333.333333,0.0,0.0,2200000.0,2000000.0,666666.666667,...,1372.0,430.5,2716.0,946.333333,200.0,80.0,223.0,861.0,5432.0,2839.0


In [45]:
sum_group2_v2.dtypes

Policy_Number                     object
車損_max_Insured_Amount1           float64
車責_max_Insured_Amount1           float64
車損_mean_Insured_Amount1          float64
車責_mean_Insured_Amount1          float64
車損_min_Insured_Amount1           float64
車損_sum_Insured_Amount1           float64
車責_sum_Insured_Amount1           float64
車責_max_Insured_Amount2           float64
車責_mean_Insured_Amount2          float64
車責_sum_Insured_Amount2           float64
竊盜_max_Insured_Amount3           float64
車損_max_Insured_Amount3           float64
車責_max_Insured_Amount3           float64
竊盜_mean_Insured_Amount3          float64
車損_mean_Insured_Amount3          float64
車責_mean_Insured_Amount3          float64
竊盜_min_Insured_Amount3           float64
車損_min_Insured_Amount3           float64
車責_min_Insured_Amount3           float64
竊盜_sum_Insured_Amount3           float64
車損_sum_Insured_Amount3           float64
車責_sum_Insured_Amount3           float64
竊盜_max_Premium                   float64
車損_max_Premium  

In [46]:
# sum_group2.to_feather('data/mdata/policy_sum_group2')
sum_group2.to_feather('data/mdata/policy_sum_group2_v2')

###### 2-3.sum_group3

In [47]:
# 改保留group1
tmp_drop_col = drop_column + group1 + use_in_group2_only

# drop columns
sum_group3 = policy_part2_flt.drop(columns=tmp_drop_col)

sum_group3.head()

,Policy_Number,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Premium_except_cancel
0,79110176bf64b5094c19aad785aeac56e36cb609,16G,2000000,0,10000000,0.0,1551,1551
1,79110176bf64b5094c19aad785aeac56e36cb609,16P,0,0,300000,0.0,1163,1163
2,61a449f6180b23379a584af904d7412205d8ff8c,16G,5000000,0,60000000,0.0,2031,2031
3,61a449f6180b23379a584af904d7412205d8ff8c,16P,0,0,500000,0.0,1098,1098
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,05N,0,0,65000,0.0,149,149


In [48]:
# pivot
sum_group3= sum_group3.pivot(index='Policy_Number', 
                             columns='Insurance_Coverage').fillna(0)

display_all(sum_group3.head(2))

Insured_Amount1                      \
Insurance_Coverage                                   00I  01A  01J  02K  03L   
Policy_Number                                                                  
000061141b237e8619efedcd6939fddeff05b9a5             0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5             0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        04M  05E  05N  06F  07P  08H  09@   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        09I  10A  12L  14E  14N  15F  15O   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                              \
Insurance_Coverage                              16G  16P  18@  18I  20B  20K   
Policy_Number                                                                  
000061141b237e8619efedcd6939fddeff05b9a5   500000.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  2000000.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        25G       26H  27I  29B  29K  32N   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0       0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  200000.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        33F  33O  34P  35H  36I  37J  40M   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        41E  41N  42F  45@  46A  47B  51O   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        55J  56B  56K  57C  57L  65K  66C   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Insurance_Coverage                        66L  67D  68E  68N  70G  70P  71H   
Policy_Number                                                                 
000061141b237e8619efedcd6939fddeff05b9a5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                              Insured_Amount2                 \
Insurance_Coverage                        72@             00I  01A  01J  02K   
Policy_Number                                                                  
000061141b237e8619efedcd6939fddeff05b9a5  0.0             0.0  0.0  0.0  0.0   
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5  0.0             0.0  0.0  0.0  0.0   

                                                                  

In [49]:
# 重設index，把policy_number拋出來
sum_group3.reset_index(inplace = True)

In [50]:
new_col_NM = []
for item in sum_group3.columns.values:
    item = item[::-1]
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[1])
    else:
        new_col_NM.append('_'.join(item))
#         print('_'.join(item))

In [51]:
sum_group3.columns = new_col_NM
display_all(sum_group3.head(2))

,Policy_Number,00I_Insured_Amount1,01A_Insured_Amount1,01J_Insured_Amount1,02K_Insured_Amount1,03L_Insured_Amount1,04M_Insured_Amount1,05E_Insured_Amount1,05N_Insured_Amount1,06F_Insured_Amount1,07P_Insured_Amount1,08H_Insured_Amount1,09@_Insured_Amount1,09I_Insured_Amount1,10A_Insured_Amount1,12L_Insured_Amount1,14E_Insured_Amount1,14N_Insured_Amount1,15F_Insured_Amount1,15O_Insured_Amount1,16G_Insured_Amount1,16P_Insured_Amount1,18@_Insured_Amount1,18I_Insured_Amount1,20B_Insured_Amount1,20K_Insured_Amount1,25G_Insured_Amount1,26H_Insured_Amount1,27I_Insured_Amount1,29B_Insured_Amount1,29K_Insured_Amount1,32N_Insured_Amount1,33F_Insured_Amount1,33O_Insured_Amount1,34P_Insured_Amount1,35H_Insured_Amount1,36I_Insured_Amount1,37J_Insured_Amount1,40M_Insured_Amount1,41E_Insured_Amount1,41N_Insured_Amount1,42F_Insured_Amount1,45@_Insured_Amount1,46A_Insured_Amount1,47B_Insured_Amount1,51O_Insured_Amount1,55J_Insured_Amount1,56B_Insured_Amount1,56K_Insured_Amount1,57C_Insured_Amount1,57L_Insured_Amount1,65K_Insured_Amount1,66C_Insured_Amount1,66L_Insured_Amount1,67D_Insured_Amount1,68E_Insured_Amount1,68N_Insured_Amount1,70G_Insured_Amount1,70P_Insured_Amount1,71H_Insured_Amount1,72@_Insured_Amount1,00I_Insured_Amount2,01A_Insured_Amount2,01J_Insured_Amount2,02K_Insured_Amount2,03L_Insured_Amount2,04M_Insured_Amount2,05E_Insured_Amount2,05N_Insured_Amount2,06F_Insured_Amount2,07P_Insured_Amount2,08H_Insured_Amount2,09@_Insured_Amount2,09I_Insured_Amount2,10A_Insured_Amount2,12L_Insured_Amount2,14E_Insured_Amount2,14N_Insured_Amount2,15F_Insured_Amount2,15O_Insured_Amount2,16G_Insured_Amount2,16P_Insured_Amount2,18@_Insured_Amount2,18I_Insured_Amount2,20B_Insured_Amount2,20K_Insured_Amount2,25G_Insured_Amount2,26H_Insured_Amount2,27I_Insured_Amount2,29B_Insured_Amount2,29K_Insured_Amount2,32N_Insured_Amount2,33F_Insured_Amount2,33O_Insured_Amount2,34P_Insured_Amount2,35H_Insured_Amount2,36I_Insured_Amount2,37J_Insured_Amount2,40M_Insured_Amount2,41E_Insured_Amount2,41N_Insured_Amount2,42F_Insured_Amount2,45@_Insured_Amount2,46A_Insured_Amount2,47B_Insured_Amount2,51O_Insured_Amount2,55J_Insured_Amount2,56B_Insured_Amount2,56K_Insured_Amount2,57C_Insured_Amount2,57L_Insured_Amount2,65K_Insured_Amount2,66C_Insured_Amount2,66L_Insured_Amount2,67D_Insured_Amount2,68E_Insured_Amount2,68N_Insured_Amount2,70G_Insured_Amount2,70P_Insured_Amount2,71H_Insured_Amount2,72@_Insured_Amount2,00I_Insured_Amount3,01A_Insured_Amount3,01J_Insured_Amount3,02K_Insured_Amount3,03L_Insured_Amount3,04M_Insured_Amount3,05E_Insured_Amount3,05N_Insured_Amount3,06F_Insured_Amount3,07P_Insured_Amount3,08H_Insured_Amount3,09@_Insured_Amount3,09I_Insured_Amount3,10A_Insured_Amount3,12L_Insured_Amount3,14E_Insured_Amount3,14N_Insured_Amount3,15F_Insured_Amount3,15O_Insured_Amount3,16G_Insured_Amount3,16P_Insured_Amount3,18@_Insured_Amount3,18I_Insured_Amount3,20B_Insured_Amount3,20K_Insured_Amount3,25G_Insured_Amount3,26H_Insured_Amount3,27I_Insured_Amount3,29B_Insured_Amount3,29K_Insured_Amount3,32N_Insured_Amount3,33F_Insured_Amount3,33O_Insured_Amount3,34P_Insured_Amount3,35H_Insured_Amount3,36I_Insured_Amount3,37J_Insured_Amount3,40M_Insured_Amount3,41E_Insured_Amount3,41N_Insured_Amount3,42F_Insured_Amount3,45@_Insured_Amount3,46A_Insured_Amount3,47B_Insured_Amount3,51O_Insured_Amount3,55J_Insured_Amount3,56B_Insured_Amount3,56K_Insured_Amount3,57C_Insured_Amount3,57L_Insured_Amount3,65K_Insured_Amount3,66C_Insured_Amount3,66L_Insured_Amount3,67D_Insured_Amount3,68E_Insured_Amount3,68N_Insured_Amount3,70G_Insured_Amount3,70P_Insured_Amount3,71H_Insured_Amount3,72@_Insured_Amount3,00I_Coverage_Deductible_if_applied,01A_Coverage_Deductible_if_applied,01J_Coverage_Deductible_if_applied,02K_Coverage_Deductible_if_applied,03L_Coverage_Deductible_if_applied,04M_Coverage_Deductible_if_applied,05E_Coverage_Deductible_if_applied,05N_Coverage_Deductible_if_applied,06F_Coverage_Deductible_if_applied,07P_Coverage_Deductible_if_applied,08H_Coverage_Deductible_if_applied,

In [52]:
filter_df = sum_group3.describe(percentiles=[0.9, 0.95, 0.99]).T
filter_df.head(2)

,count,mean,std,min,50%,90%,95%,99%,max
00I_Insured_Amount1,351273.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
01A_Insured_Amount1,351273.0,25.251016,788.061021,0.0,0.0,0.0,0.0,0.0,60000.0


In [53]:
flt = (filter_df['90%'] == 0)
sum(flt)

325

In [54]:
drop_NM = filter_df[flt].index.values
sum_group3_v2 = sum_group3.drop(columns=drop_NM)
sum_group3_v2.head(2)

,Policy_Number,16G_Insured_Amount1,18@_Insured_Amount1,20K_Insured_Amount1,18@_Insured_Amount2,29B_Insured_Amount2,04M_Insured_Amount3,05E_Insured_Amount3,05N_Insured_Amount3,09@_Insured_Amount3,...,04M_Premium_except_cancel,05E_Premium_except_cancel,05N_Premium_except_cancel,09@_Premium_except_cancel,16G_Premium_except_cancel,16P_Premium_except_cancel,18@_Premium_except_cancel,20K_Premium_except_cancel,29B_Premium_except_cancel,29K_Premium_except_cancel
0,000061141b237e8619efedcd6939fddeff05b9a5,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,660.0,727.0,0.0,0.0,0.0,199.0
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,2000000.0,0.0,0.0,0.0,0.0,272000.0,272000.0,272000.0,0.0,...,5352.0,80.0,661.0,0.0,1372.0,1244.0,0.0,0.0,0.0,0.0


In [55]:
# sum_group3.to_feather('data/mdata/policy_sum_group3')
sum_group3.to_feather('data/mdata/policy_sum_group3_v2')

### 3. 整理車子相關Features

In [56]:
# 讀取屬性檔RawData
policy_df = pd.read_csv('data/policy_0702.csv')

In [57]:
# Look, 保單跟車子1:N
# len(policy_df[['Policy_Number', 'Vehicle_identifier']].drop_duplicates())
# len(policy_df[['Vehicle_identifier']].drop_duplicates())

In [58]:
keep_col = ['Policy_Number', 'Vehicle_identifier', 'Vehicle_Make_and_Model1',
            'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
            'Engine_Displacement_(Cubic_Centimeter)',
            'Imported_or_Domestic_Car', 'Coding_of_Vehicle_Branding_&_Type',
            'qpt', 'fpt', 'fequipment1',
            'fequipment2', 'fequipment3', 'fequipment4', 'fequipment5',
            'fequipment6', 'fequipment9', 'nequipment9']

In [59]:
car_df = policy_df.loc[:, keep_col].drop_duplicates().reset_index(drop=True)

In [60]:
display_all(car_df.head(2))

,Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,0,0,0,0,0,0,0,
1,61a449f6180b23379a584af904d7412205d8ff8c,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,0,0,0,0,0,0,0,


1. fpt(乘載單位)、fequipment4、fequipment6沒什麼意義拿掉，全部都是P
2. 廠牌2跟車系先保留，分類太多太雜了
1. 車子廠牌類型過多，只先對廠牌1做處理(保留前11個，彙整後面全部為另一個)
3. 新增一欄車齡，假定資料時間為2016
5. 排氣量感覺可以轉換很多欄位，先轉成年繳的稅額看看

In [61]:
# 1.沒用的欄位拿掉
car_df.drop(columns=['fpt', 'fequipment4', 'fequipment6',
                     'fequipment1', 'fequipment2', 'fequipment3',
                     'fequipment5', 'fequipment9', 'nequipment9'
                    ], inplace=True)

In [62]:
car_df.head(2)

,Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt
0,79110176bf64b5094c19aad785aeac56e36cb609,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0
1,61a449f6180b23379a584af904d7412205d8ff8c,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0


In [63]:
# 數字轉類別
car_df.Imported_or_Domestic_Car = car_df.Imported_or_Domestic_Car.apply(str)

In [64]:
# 3. 車子廠牌類型過多，只先對廠牌1做處理(保留前11個，彙整後面全部為另一個)
# top11 = car_df.Vehicle_Make_and_Model1.value_counts()[:7]
# top11

In [65]:
# change pd values from list
# car_df.Vehicle_Make_and_Model1 = car_df.Vehicle_Make_and_Model1.apply(
#     lambda x: x if x in top11 else 'other_Vehicle_Make')

In [66]:
# look
# car_df.Vehicle_Make_and_Model1.value_counts()

In [67]:
# 4. 新增一欄車齡，假定資料時間為2016
car_df['car_age'] = (2016 - car_df.Manafactured_Year_and_Month)

In [68]:
# look
# car_df.car_age.value_counts()

In [69]:
# 5. 排氣量感覺可以轉換很多欄位，先轉成年繳的稅額看看
# tax1 = 牌照稅
car_df['tax_1'] = 0
car_df['tax_2'] = 0

In [70]:
tax1_eng_range = [0, 500, 600, 1200, 1800, 2400, 3000, 4200, 5400, 6600, 7800]
tax1_amt = [1620, 2160, 4320, 7120, 11230, 15210, 28220, 46170, 69690, 117000, 151200]
tax1 = np.stack([tax1_eng_range, tax1_amt], axis=1)

for i in tax1:
    flt_1 = (car_df['Engine_Displacement_(Cubic_Centimeter)'] > i[0])
    car_df.loc[flt_1, ['tax_1']] = i[1]

In [71]:
tax2_eng_range = [250, 500, 600, 1200, 1800, 2400, 3000, 3600, 4200, 4800, 5400, 6000, 6600, 7200]
tax2_amt = [2160, 2880, 4320, 4800, 6210, 7200, 8640, 9810, 11220, 12180, 13080, 13950, 14910, 15720]
tax2 = np.stack([tax2_eng_range, tax2_amt], axis=1)

for i in tax2:
    flt_1 = (car_df['Engine_Displacement_(Cubic_Centimeter)'] > i[0])
    car_df.loc[flt_1, ['tax_2']] = i[1]

In [72]:
# look
car_df.head(2)

,Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,car_age,tax_1,tax_2
0,79110176bf64b5094c19aad785aeac56e36cb609,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,6,7120,4800
1,61a449f6180b23379a584af904d7412205d8ff8c,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,10,7120,4800


In [73]:
car_df.dtypes

Policy_Number                              object
Vehicle_identifier                         object
Vehicle_Make_and_Model1                    object
Vehicle_Make_and_Model2                    object
Manafactured_Year_and_Month                 int64
Engine_Displacement_(Cubic_Centimeter)      int64
Imported_or_Domestic_Car                   object
Coding_of_Vehicle_Branding_&_Type          object
qpt                                       float64
car_age                                     int64
tax_1                                       int64
tax_2                                       int64
dtype: object

In [74]:
# car_df.to_feather('data/mdata/car')
car_df.to_feather('data/mdata/car_v2')

### 4. 整理保人相關Features (含理賠紀錄)

強制汽車責任保險保費依照個人之年齡性別及前三年有無肇事記錄而有不同之保費。

In [75]:
# 讀取屬性檔RawData
policy_df = pd.read_csv('data/policy_0702.csv')
claim_df = pd.read_csv('data/claim_0702.csv')

In [76]:
keep_col = ['Policy_Number', "Insured's_ID", 'fassured', 'ibirth', 'dbirth', 
            'fsex', 'fmarriage', 'aassured_zip', 'iply_area']

In [77]:
man_df = policy_df.loc[:, keep_col].drop_duplicates().reset_index(drop=True)

1. fassured(被保人性質)，把6換成2(法人Key錯換成法人)
2. ibirth沒辦法parser成日期，手動拆成年、月、還有年齡三欄
3. dbirth(要保人生日)，小於18歲做成有小孩的註記，另外很多超過2018年的錯誤資料，手動修正 (x, 小於18歲的太少)，改做(被保人 = 要保人)註記
4. aassured_zip(承保公司代號)屬於high cardinality category，暫略

In [78]:
# 1. fassured(被保人性質)，把6換成2(法人Key錯換成法人)
tmp_flt = (man_df.fassured == 6)
man_df.loc[tmp_flt, 'fassured'] = 2

In [79]:
# 2. ibirth沒辦法parser成日期，自己手動拆成年、月、還有年齡三欄
man_df['ins_year'] = man_df.ibirth.str[3:].fillna('0')
man_df['ins_mon'] = man_df.ibirth.str[0:2].fillna('0')

In [80]:
# 轉換類型object -> int
man_df.ins_year = man_df.ins_year.astype(str).astype(int)
man_df.ins_mon = man_df.ins_mon.astype(str).astype(int)

In [81]:
man_df['age'] = (2016 - man_df.ins_year)
man_df.age = man_df.age.apply(lambda x: 0 if x > 100 else x)

In [82]:
# 3. dbirth(要保人生日)，改做(被保人 = 要保人)註記
flt = (man_df.ibirth == man_df.dbirth)
man_df['di_equal'] = 0
man_df.loc[flt, ['di_equal']] = 1

In [83]:
man_df.head(2)

,Policy_Number,Insured's_ID,fassured,ibirth,dbirth,fsex,fmarriage,aassured_zip,iply_area,ins_year,ins_mon,age,di_equal
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,1,08/1984,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,1984,8,32,1
1,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,1,10/1975,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,1975,10,41,1


In [84]:
man_df.dtypes

Policy_Number    object
Insured's_ID     object
fassured          int64
ibirth           object
dbirth           object
fsex             object
fmarriage        object
aassured_zip     object
iply_area        object
ins_year          int64
ins_mon           int64
age               int64
di_equal          int64
dtype: object

In [85]:
man_df.to_feather('data/mdata/man')

#### 理賠紀錄(待++)

In [86]:
!head -2 data/claim_0702.csv

Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00


In [87]:
claim_df.head(10)

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00
5,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,41077,0,0,16G,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00
6,40e4d486a1ae4d46095b4a7620c32efc365adf44,1,9f0c0a423c90a4bf99fb797c8160b85366269702,1,1,10/1959,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,e9f0adcfc3f458f292a0347ee321335b76186b78,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.787,22:00
7,ccd799c50d0abef95ba1a953db797835e9572acd,1,1cc72e143a54f8572a9c7a0dd54dc835925b46ae,1,3,05/1985,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,14000,0,0,16P,874d549ec9ebef3fb061da572d540a011440cdf2,100.0,1,0,c66c65175fecc3103b3b587be9b5b230889c8628,1.981,09:00
8,ccd799c50d0abef95ba1a953db797835e9572acd,1,1cc72e143a54f8572a9c7a0dd54dc835925b46ae,1,3,05/1985,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,700,0,16P,874d549ec9ebef3fb061da572d540a011440cdf2,100.0,1,0,c66c65175fecc3103b3b587be9b5b230889c8628,1.981,09:00
9,ccd799c50d0abef95ba1a953db797835e9572acd,1,1cc72e143a54f8572a9c7a0dd54dc835925b46ae,1,3,05/1985,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1000,0,0,16G,874d549ec9ebef3fb061da572d540a011440cdf2,100.0,1,0,c66c65175fecc3103b3b587be9b5b230889c8628,1.981,09:00


拿掉Coverage <br>
加總下列4欄：Paid_Loss_Amount	paid_Expenses_Amount	Salvage_or_Subrogation?	 Deductible

In [88]:
claim_no_dup = claim_df.drop(columns=['Coverage', 'Paid_Loss_Amount', 'paid_Expenses_Amount', 'Salvage_or_Subrogation?', 'Deductible']).drop_duplicates()
claim_no_dup.head()

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Accident_area,number_of_claimants,Accident_Time
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00
6,40e4d486a1ae4d46095b4a7620c32efc365adf44,1,9f0c0a423c90a4bf99fb797c8160b85366269702,1,1,10/1959,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,e9f0adcfc3f458f292a0347ee321335b76186b78,100.0,1,356a192b7913b04c54574d18c28d46e6395428ab,1.787,22:00
7,ccd799c50d0abef95ba1a953db797835e9572acd,1,1cc72e143a54f8572a9c7a0dd54dc835925b46ae,1,3,05/1985,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,874d549ec9ebef3fb061da572d540a011440cdf2,100.0,1,c66c65175fecc3103b3b587be9b5b230889c8628,1.981,09:00


In [89]:
claim_df[claim_df.Policy_Number == '2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86']

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
16252,08983057eeec099f6b37ff17d162bd248e894f89,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/06,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,04M,c675f137a5388d436314f4b9547559f3596899e8,100.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,23:00
16253,08983057eeec099f6b37ff17d162bd248e894f89,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/06,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,05E,c675f137a5388d436314f4b9547559f3596899e8,100.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,23:00
16254,08983057eeec099f6b37ff17d162bd248e894f89,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/06,17ba0791499db908433b80f37c5fbc89b870084b,9390,0,0,16P,c675f137a5388d436314f4b9547559f3596899e8,100.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,23:00
17610,346835c76557252f1b90826667230e341e884bc3,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,04M,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
17611,346835c76557252f1b90826667230e341e884bc3,2,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,04M,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
17612,346835c76557252f1b90826667230e341e884bc3,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,18714,0,0,05E,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
17613,346835c76557252f1b90826667230e341e884bc3,2,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,11000,05E,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
17614,346835c76557252f1b90826667230e341e884bc3,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
17615,346835c76557252f1b90826667230e341e884bc3,2,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,c675f137a5388d436314f4b9547559f3596899e8,30.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,13:00
18190,72f764116423deb19f5f5e4485cd411938b0b50c,1,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,2,5,01/1978,1,2016/06,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,04M,c675f137a5388d436314f4b9547559f3596899e8,100.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,1.981,22:00


(v0825): 理賠紀錄蠻難處理的，只保留Policy_Number的賠案數

In [90]:
clm_v0 = claim_df[['Policy_Number', 'Claim_Number']].drop_duplicates().reset_index(drop=True)
clm_v0.head()

,Policy_Number,Claim_Number
0,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,954ff50c11e2c6f6bf91514fdcca98578128da21
1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,e33ba1121d6772dd84e83b266d66b1a7dea26d71
2,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,f26827172ff9528d11c66868cbf57f4e9c2d537c
3,9f0c0a423c90a4bf99fb797c8160b85366269702,40e4d486a1ae4d46095b4a7620c32efc365adf44
4,1cc72e143a54f8572a9c7a0dd54dc835925b46ae,ccd799c50d0abef95ba1a953db797835e9572acd


In [91]:
clm_df = clm_v0.groupby(by='Policy_Number', as_index=False).count()
clm_df.columns = ['Policy_Number', 'N_of_Claim']
clm_df.head()

,Policy_Number,N_of_Claim
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,1
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,1
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,2
3,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,1
4,000a5d472fa501c2e9d771dfae4af646f281f97d,1


In [92]:
clm_df.dtypes

Policy_Number    object
N_of_Claim        int64
dtype: object

In [93]:
clm_df.to_feather('data/mdata/clm')

### 5. 整理分公司相關Features

In [94]:
# 讀取屬性檔RawData
policy_df = pd.read_csv('data/policy_0702.csv')

In [95]:
effective_df = policy_df[policy_df.Coverage_Deductible_if_applied >= 0]
effective_df.shape, policy_df.shape

((1741799, 41), (1747942, 41))

#### 對分公司直接count

In [96]:
cnt_col = ['iply_area', "Insured's_ID", 'Policy_Number',
           'Vehicle_identifier', 'Distribution_Channel'
          ]

In [97]:
area_p1 = effective_df[cnt_col].drop_duplicates().reset_index(drop=True)
print(area_p1.shape)

(351257, 5)


In [98]:
area_attr1 = area_p1.groupby(by="iply_area").nunique().drop(columns='iply_area')
area_attr1.columns = 'area_' + area_attr1.columns.values

area_attr1['avg_ply_per_customer'] = area_attr1.area_Policy_Number / area_attr1["area_Insured's_ID"]
area_attr1['avg_car_per_customer'] = area_attr1.area_Vehicle_identifier / area_attr1["area_Insured's_ID"]
area_attr1['avg_customer_per_channel'] = area_attr1["area_Insured's_ID"] / area_attr1.area_Distribution_Channel
area_attr1['avg_ply_per_channel'] = area_attr1.area_Policy_Number / area_attr1.area_Distribution_Channel

area_attr1.reset_index(drop = False, inplace = True)

In [99]:
print(area_attr1.shape)
area_attr1.head(2)

(22, 9)


,iply_area,area_Insured's_ID,area_Policy_Number,area_Vehicle_identifier,area_Distribution_Channel,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_channel,avg_ply_per_channel
0,0286dd552c9bea9a69ecb3759e7b94777635514b,3810,4407,4379,141,1.156693,1.149344,27.021277,31.255319
1,17ba0791499db908433b80f37c5fbc89b870084b,5027,5676,5618,153,1.129103,1.117565,32.856209,37.098039


In [100]:
area_attr1.dtypes

iply_area                     object
area_Insured's_ID              int64
area_Policy_Number             int64
area_Vehicle_identifier        int64
area_Distribution_Channel      int64
avg_ply_per_customer         float64
avg_car_per_customer         float64
avg_customer_per_channel     float64
avg_ply_per_channel          float64
dtype: object

#### 對分公司直接sum

In [101]:
cnt_col = ['iply_area', 'Premium']

area_p2 = effective_df[cnt_col]
print(area_p2.shape); area_p2.head(3)

(1741799, 2)


,iply_area,Premium
0,c1dfd96eea8cc2b62785275bca38ac261256e278,1551
1,c1dfd96eea8cc2b62785275bca38ac261256e278,1163
2,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,2031


In [102]:
area_attr2 = area_p2.groupby(by = 'iply_area').agg(['sum', 'max', 'min', 'mean'])
print(area_attr2.shape)
area_attr2.head(2)

(22, 4)


Premium                         
                                               sum     max min         mean
iply_area                                                                  
0286dd552c9bea9a69ecb3759e7b94777635514b  26471361  288961   3  1163.883266
17ba0791499db908433b80f37c5fbc89b870084b  31688584   85926   0  1260.885883

In [103]:
new_col_NM = []
for item in area_attr2.columns.values:
    item = item[::-1] #保留except_cancel，後面資料比較好處理
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[2])
    else:
        new_col_NM.append('area_' + '_'.join(item))
#         print('_'.join(item))

# new_col_NM
area_attr2.columns = new_col_NM
area_attr2.reset_index(drop = False, inplace = True)
print(area_attr2.shape); area_attr2.head(2)

(22, 5)


,iply_area,area_sum_Premium,area_max_Premium,area_min_Premium,area_mean_Premium
0,0286dd552c9bea9a69ecb3759e7b94777635514b,26471361,288961,3,1163.883266
1,17ba0791499db908433b80f37c5fbc89b870084b,31688584,85926,0,1260.885883


In [104]:
area_attr2.dtypes

iply_area             object
area_sum_Premium       int64
area_max_Premium       int64
area_min_Premium       int64
area_mean_Premium    float64
dtype: object

#### 對分公司保單做計算

In [105]:
attr3_col = ['iply_area', 
             'Cancellation', 'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
             'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3', 
             'Coverage_Deductible_if_applied', 'Replacement_cost_of_insured_vehicle'
            ]
seg_col = ['Cancellation', 'Main_Insurance_Coverage_Group', 'Insurance_Coverage']

In [106]:
attr3_df = effective_df[attr3_col]

In [107]:
# 特定險種代號的自負額欄位值與自負額無關，改設為0
Insurance_Coverage_no_cov = ['09I', '10A', '14E', '15F', 
                             '15O', '20B', '20K', '29K', 
                             '32N', '33F', '33O', '56K', 
                             '65K']
flt = attr3_df['Insurance_Coverage'].isin(Insurance_Coverage_no_cov)
idx = attr3_df[flt].index

# It's ok
attr3_df.loc[idx, 'Coverage_Deductible_if_applied'] = 0

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [108]:
area_attr3 = attr3_df.groupby(by='iply_area').agg(['sum', 'mean', 'max', 'min']).reset_index()
area_attr3.head(2)

iply_area Insured_Amount1                 \
                                                        sum           mean   
0  0286dd552c9bea9a69ecb3759e7b94777635514b     10136336000  445670.770313   
1  17ba0791499db908433b80f37c5fbc89b870084b     12183819000  484793.052682   

                Insured_Amount2                              Insured_Amount3  \
        max min             sum           mean       max min             sum   
0  15000000   0      9327950000  410127.945832  18000000   0    118963435500   
1  20000000   0      6481880000  257913.417157  18000000   0    117172242500   

   ...                 Coverage_Deductible_if_applied                        \
   ...         max min                            sum       mean    max min   
0  ...   120000000   0                         368466  16.200580  50000   0   
1  ...   140000000   0                         252541  10.048583  30000   0   

  Replacement_cost_of_insured_vehicle                           
                                  sum       mean     max   min  
0                           1995858.4  87.753183  2300.0  16.0  
1                           2152989.3  85.667249  1200.0   1.0  

[2 rows x 21 columns]

In [109]:
new_col_NM = []
for item in area_attr3.columns.values:
    item = item[::-1] #保留except_cancel，後面資料比較好處理
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[2])
    else:
        new_col_NM.append('area_' + '_'.join(item))
#         print('_'.join(item))

# new_col_NM
area_attr3.columns = new_col_NM
print(area_attr3.shape); area_attr3.head(2)

(22, 21)


,iply_area,area_sum_Insured_Amount1,area_mean_Insured_Amount1,area_max_Insured_Amount1,area_min_Insured_Amount1,area_sum_Insured_Amount2,area_mean_Insured_Amount2,area_max_Insured_Amount2,area_min_Insured_Amount2,area_sum_Insured_Amount3,...,area_max_Insured_Amount3,area_min_Insured_Amount3,area_sum_Coverage_Deductible_if_applied,area_mean_Coverage_Deductible_if_applied,area_max_Coverage_Deductible_if_applied,area_min_Coverage_Deductible_if_applied,area_sum_Replacement_cost_of_insured_vehicle,area_mean_Replacement_cost_of_insured_vehicle,area_max_Replacement_cost_of_insured_vehicle,area_min_Replacement_cost_of_insured_vehicle
0,0286dd552c9bea9a69ecb3759e7b94777635514b,10136336000,445670.770313,15000000,0,9327950000,410127.945832,18000000,0,118963435500,...,120000000,0,368466,16.200580,50000,0,1995858.4,87.753183,2300.0,16.0
1,17ba0791499db908433b80f37c5fbc89b870084b,12183819000,484793.052682,20000000,0,6481880000,257913.417157,18000000,0,117172242500,...,140000000,0,252541,10.048583,30000,0,2152989.3,85.667249,1200.0,1.0


In [110]:
area_attr3.dtypes

iply_area                                         object
area_sum_Insured_Amount1                           int64
area_mean_Insured_Amount1                        float64
area_max_Insured_Amount1                           int64
area_min_Insured_Amount1                           int64
area_sum_Insured_Amount2                           int64
area_mean_Insured_Amount2                        float64
area_max_Insured_Amount2                           int64
area_min_Insured_Amount2                           int64
area_sum_Insured_Amount3                           int64
area_mean_Insured_Amount3                        float64
area_max_Insured_Amount3                           int64
area_min_Insured_Amount3                           int64
area_sum_Coverage_Deductible_if_applied            int64
area_mean_Coverage_Deductible_if_applied         float64
area_max_Coverage_Deductible_if_applied            int64
area_min_Coverage_Deductible_if_applied            int64
area_sum_Replacement_cost_of_in

#### 彙整計算

In [111]:
area_attr = area_attr1.merge(area_attr2, on='iply_area')
area_attr = area_attr.merge(area_attr3, on='iply_area')

print(area_attr.shape); area_attr.head(2)

(22, 33)


,iply_area,area_Insured's_ID,area_Policy_Number,area_Vehicle_identifier,area_Distribution_Channel,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_channel,avg_ply_per_channel,area_sum_Premium,...,area_max_Insured_Amount3,area_min_Insured_Amount3,area_sum_Coverage_Deductible_if_applied,area_mean_Coverage_Deductible_if_applied,area_max_Coverage_Deductible_if_applied,area_min_Coverage_Deductible_if_applied,area_sum_Replacement_cost_of_insured_vehicle,area_mean_Replacement_cost_of_insured_vehicle,area_max_Replacement_cost_of_insured_vehicle,area_min_Replacement_cost_of_insured_vehicle
0,0286dd552c9bea9a69ecb3759e7b94777635514b,3810,4407,4379,141,1.156693,1.149344,27.021277,31.255319,26471361,...,120000000,0,368466,16.200580,50000,0,1995858.4,87.753183,2300.0,16.0
1,17ba0791499db908433b80f37c5fbc89b870084b,5027,5676,5618,153,1.129103,1.117565,32.856209,37.098039,31688584,...,140000000,0,252541,10.048583,30000,0,2152989.3,85.667249,1200.0,1.0


In [112]:
area_attr['Insured_Amount1_per_Channel'] = area_attr.area_sum_Insured_Amount1 / area_attr.area_Distribution_Channel
area_attr['Insured_Amount2_per_Channel'] = area_attr.area_sum_Insured_Amount2 / area_attr.area_Distribution_Channel
area_attr['Insured_Amount3_per_Channel'] = area_attr.area_sum_Insured_Amount3 / area_attr.area_Distribution_Channel
area_attr['Coverage_Deductible_if_applied_per_Channel'] = area_attr.area_sum_Coverage_Deductible_if_applied / area_attr.area_Distribution_Channel
area_attr['Replacement_cost_of_insured_vehicle_per_Channel'] = area_attr.area_sum_Replacement_cost_of_insured_vehicle / area_attr.area_Distribution_Channel
area_attr['Premium_per_Channel_per_Channel'] = area_attr.area_sum_Premium / area_attr.area_Distribution_Channel

In [113]:
area_attr['Insured_Amount1_per_ply'] = area_attr.area_sum_Insured_Amount1 / area_attr.area_Policy_Number
area_attr['Insured_Amount2_per_ply'] = area_attr.area_sum_Insured_Amount2 / area_attr.area_Policy_Number
area_attr['Insured_Amount3_per_ply'] = area_attr.area_sum_Insured_Amount3 / area_attr.area_Policy_Number
area_attr['Coverage_Deductible_if_applied_per_ply'] = area_attr.area_sum_Coverage_Deductible_if_applied / area_attr.area_Policy_Number
area_attr['Replacement_cost_of_insured_vehicle_per_ply'] = area_attr.area_sum_Replacement_cost_of_insured_vehicle / area_attr.area_Policy_Number
area_attr['Premium_per_ply_per_ply'] = area_attr.area_sum_Premium / area_attr.area_Policy_Number

In [114]:
area_attr['Insured_Amount1_per_Customer'] = area_attr.area_sum_Insured_Amount1 / area_attr["area_Insured's_ID"]
area_attr['Insured_Amount2_per_Customer'] = area_attr.area_sum_Insured_Amount2 / area_attr["area_Insured's_ID"]
area_attr['Insured_Amount3_per_Customer'] = area_attr.area_sum_Insured_Amount3 / area_attr["area_Insured's_ID"]
area_attr['Coverage_Deductible_if_applied_per_Customer'] = area_attr.area_sum_Coverage_Deductible_if_applied / area_attr["area_Insured's_ID"]
area_attr['Replacement_cost_of_insured_vehicle_per_Customer'] = area_attr.area_sum_Replacement_cost_of_insured_vehicle / area_attr["area_Insured's_ID"]
area_attr['Premium_per_Customer_per_Customer'] = area_attr.area_sum_Premium / area_attr["area_Insured's_ID"]

In [115]:
print(area_attr.shape); area_attr.head(2)

(22, 51)


,iply_area,area_Insured's_ID,area_Policy_Number,area_Vehicle_identifier,area_Distribution_Channel,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_channel,avg_ply_per_channel,area_sum_Premium,...,Insured_Amount3_per_ply,Coverage_Deductible_if_applied_per_ply,Replacement_cost_of_insured_vehicle_per_ply,Premium_per_ply_per_ply,Insured_Amount1_per_Customer,Insured_Amount2_per_Customer,Insured_Amount3_per_Customer,Coverage_Deductible_if_applied_per_Customer,Replacement_cost_of_insured_vehicle_per_Customer,Premium_per_Customer_per_Customer
0,0286dd552c9bea9a69ecb3759e7b94777635514b,3810,4407,4379,141,1.156693,1.149344,27.021277,31.255319,26471361,...,2.699420e+07,83.609258,452.883685,6006.662355,2.660456e+06,2.448281e+06,3.122400e+07,96.710236,523.847349,6947.863780
1,17ba0791499db908433b80f37c5fbc89b870084b,5027,5676,5618,153,1.129103,1.117565,32.856209,37.098039,31688584,...,2.064345e+07,44.492777,379.314535,5582.907681,2.423676e+06,1.289413e+06,2.330858e+07,50.236921,428.285120,6303.676944


In [116]:
area_attr.to_feather('data/mdata/area_attr')

### 6. 整理通路相關Features

In [117]:
# 讀取屬性檔RawData
policy_df = pd.read_csv('data/policy_0702.csv')

In [118]:
effective_df = policy_df[policy_df.Coverage_Deductible_if_applied >= 0]
effective_df.shape, policy_df.shape

((1741799, 41), (1747942, 41))

#### 對分公司直接count

In [119]:
cnt_col = ['iply_area', "Insured's_ID", 'Policy_Number',
           'Vehicle_identifier', 'Distribution_Channel'
          ]

In [120]:
channel_p1 = effective_df[cnt_col].drop_duplicates().reset_index(drop=True)
print(channel_p1.shape)

(351257, 5)


In [121]:
channel_attr1 = channel_p1.groupby(by="Distribution_Channel").nunique().drop(columns='Distribution_Channel')
channel_attr1.columns = 'channel_' + channel_attr1.columns.values

channel_attr1['avg_ply_per_customer'] = channel_attr1.channel_Policy_Number / channel_attr1["channel_Insured's_ID"]
channel_attr1['avg_car_per_customer'] = channel_attr1.channel_Vehicle_identifier / channel_attr1["channel_Insured's_ID"]
channel_attr1['avg_customer_per_area'] = channel_attr1["channel_Insured's_ID"] / channel_attr1.channel_iply_area
channel_attr1['avg_ply_per_area'] = channel_attr1.channel_Policy_Number / channel_attr1.channel_iply_area

channel_attr1.reset_index(drop = False, inplace = True)

In [122]:
print(channel_attr1.shape)
channel_attr1.head(2)

(875, 9)


,Distribution_Channel,channel_iply_area,channel_Insured's_ID,channel_Policy_Number,channel_Vehicle_identifier,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_area,avg_ply_per_area
0,00520d090d60ddb041b1c7043d77d5073bdcd464,2,2,2,2,1.000000,1.000000,1.0,1.000000
1,0090902477ffe105c4b342a37987a0b82103e7b0,15,360,388,382,1.077778,1.061111,24.0,25.866667


In [123]:
channel_attr1.dtypes

Distribution_Channel           object
channel_iply_area               int64
channel_Insured's_ID            int64
channel_Policy_Number           int64
channel_Vehicle_identifier      int64
avg_ply_per_customer          float64
avg_car_per_customer          float64
avg_customer_per_area         float64
avg_ply_per_area              float64
dtype: object

#### 對分公司直接sum

In [124]:
cnt_col = ['Distribution_Channel', 'Premium']

channel_p2 = effective_df[cnt_col]
print(channel_p2.shape); channel_p2.head(3)

(1741799, 2)


,Distribution_Channel,Premium
0,5037717f120cf25bbeea6a42e62676bc5c911b12,1551
1,5037717f120cf25bbeea6a42e62676bc5c911b12,1163
2,5037717f120cf25bbeea6a42e62676bc5c911b12,2031


In [125]:
channel_attr2 = channel_p2.groupby(by = 'Distribution_Channel').agg(['sum', 'max', 'min', 'mean'])
print(channel_attr2.shape)
channel_attr2.head(2)

(875, 4)


Premium                         
                                              sum    max  min         mean
Distribution_Channel                                                      
00520d090d60ddb041b1c7043d77d5073bdcd464     6526   1560  199   932.285714
0090902477ffe105c4b342a37987a0b82103e7b0  2459616  40955    4  1374.087151

In [126]:
new_col_NM = []
for item in channel_attr2.columns.values:
    item = item[::-1] #保留except_cancel，後面資料比較好處理
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[2])
    else:
        new_col_NM.append('channel_' + '_'.join(item))
#         print('_'.join(item))

# new_col_NM
channel_attr2.columns = new_col_NM
channel_attr2.reset_index(drop = False, inplace = True)
print(channel_attr2.shape); channel_attr2.head(2)

(875, 5)


,Distribution_Channel,channel_sum_Premium,channel_max_Premium,channel_min_Premium,channel_mean_Premium
0,00520d090d60ddb041b1c7043d77d5073bdcd464,6526,1560,199,932.285714
1,0090902477ffe105c4b342a37987a0b82103e7b0,2459616,40955,4,1374.087151


In [127]:
channel_attr2.dtypes

Distribution_Channel     object
channel_sum_Premium       int64
channel_max_Premium       int64
channel_min_Premium       int64
channel_mean_Premium    float64
dtype: object

#### 對分公司保單做計算

In [128]:
attr3_col = ['Distribution_Channel', 
             'Cancellation', 'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
             'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3', 
             'Coverage_Deductible_if_applied', 'Replacement_cost_of_insured_vehicle'
            ]
seg_col = ['Cancellation', 'Main_Insurance_Coverage_Group', 'Insurance_Coverage']

In [129]:
attr3_df = effective_df[attr3_col]

In [130]:
# 特定險種代號的自負額欄位值與自負額無關，改設為0
Insurance_Coverage_no_cov = ['09I', '10A', '14E', '15F', 
                             '15O', '20B', '20K', '29K', 
                             '32N', '33F', '33O', '56K', 
                             '65K']
flt = attr3_df['Insurance_Coverage'].isin(Insurance_Coverage_no_cov)
idx = attr3_df[flt].index

# It's ok
attr3_df.loc[idx, 'Coverage_Deductible_if_applied'] = 0

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [131]:
channel_attr3 = attr3_df.groupby(by='Distribution_Channel').agg(['sum', 'mean', 'max', 'min']).reset_index()
channel_attr3.head(2)

Distribution_Channel Insured_Amount1                 \
                                                        sum           mean   
0  00520d090d60ddb041b1c7043d77d5073bdcd464         6200000  885714.285714   
1  0090902477ffe105c4b342a37987a0b82103e7b0      1170332500  653817.039106   

                Insured_Amount2                              Insured_Amount3  \
        max min             sum           mean       max min             sum   
0   3000000   0         3000000  428571.428571   2000000   0        21700000   
1  10000000   0       872800000  487597.765363  16000000   0     12536447000   

   ...                    Coverage_Deductible_if_applied                    \
   ...        max     min                            sum      mean max min   
0  ...    6000000  200000                              0  0.000000   0   0   
1  ...   80300000       0                           1185  0.662011  20   0   

  Replacement_cost_of_insured_vehicle                           
                                  sum        mean    max   min  
0                               753.7  107.671429  127.0  81.9  
1                            163924.8   91.578101  455.0  25.3  

[2 rows x 21 columns]

In [132]:
new_col_NM = []
for item in channel_attr3.columns.values:
    item = item[::-1] #保留except_cancel，後面資料比較好處理
    if item[0] == '':
        new_col_NM.append(item[1])
#         print(item[2])
    else:
        new_col_NM.append('channel_' + '_'.join(item))
#         print('_'.join(item))

# new_col_NM
channel_attr3.columns = new_col_NM
print(channel_attr3.shape); channel_attr3.head(2)

(875, 21)


,Distribution_Channel,channel_sum_Insured_Amount1,channel_mean_Insured_Amount1,channel_max_Insured_Amount1,channel_min_Insured_Amount1,channel_sum_Insured_Amount2,channel_mean_Insured_Amount2,channel_max_Insured_Amount2,channel_min_Insured_Amount2,channel_sum_Insured_Amount3,...,channel_max_Insured_Amount3,channel_min_Insured_Amount3,channel_sum_Coverage_Deductible_if_applied,channel_mean_Coverage_Deductible_if_applied,channel_max_Coverage_Deductible_if_applied,channel_min_Coverage_Deductible_if_applied,channel_sum_Replacement_cost_of_insured_vehicle,channel_mean_Replacement_cost_of_insured_vehicle,channel_max_Replacement_cost_of_insured_vehicle,channel_min_Replacement_cost_of_insured_vehicle
0,00520d090d60ddb041b1c7043d77d5073bdcd464,6200000,885714.285714,3000000,0,3000000,428571.428571,2000000,0,21700000,...,6000000,200000,0,0.000000,0,0,753.7,107.671429,127.0,81.9
1,0090902477ffe105c4b342a37987a0b82103e7b0,1170332500,653817.039106,10000000,0,872800000,487597.765363,16000000,0,12536447000,...,80300000,0,1185,0.662011,20,0,163924.8,91.578101,455.0,25.3


In [133]:
channel_attr3.dtypes

Distribution_Channel                                 object
channel_sum_Insured_Amount1                           int64
channel_mean_Insured_Amount1                        float64
channel_max_Insured_Amount1                           int64
channel_min_Insured_Amount1                           int64
channel_sum_Insured_Amount2                           int64
channel_mean_Insured_Amount2                        float64
channel_max_Insured_Amount2                           int64
channel_min_Insured_Amount2                           int64
channel_sum_Insured_Amount3                           int64
channel_mean_Insured_Amount3                        float64
channel_max_Insured_Amount3                           int64
channel_min_Insured_Amount3                           int64
channel_sum_Coverage_Deductible_if_applied            int64
channel_mean_Coverage_Deductible_if_applied         float64
channel_max_Coverage_Deductible_if_applied            int64
channel_min_Coverage_Deductible_if_appli

#### 彙整計算

In [134]:
channel_attr = channel_attr1.merge(channel_attr2, on='Distribution_Channel')
channel_attr = channel_attr.merge(channel_attr3, on='Distribution_Channel')

print(channel_attr.shape); channel_attr.head(2)

(875, 33)


,Distribution_Channel,channel_iply_area,channel_Insured's_ID,channel_Policy_Number,channel_Vehicle_identifier,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_area,avg_ply_per_area,channel_sum_Premium,...,channel_max_Insured_Amount3,channel_min_Insured_Amount3,channel_sum_Coverage_Deductible_if_applied,channel_mean_Coverage_Deductible_if_applied,channel_max_Coverage_Deductible_if_applied,channel_min_Coverage_Deductible_if_applied,channel_sum_Replacement_cost_of_insured_vehicle,channel_mean_Replacement_cost_of_insured_vehicle,channel_max_Replacement_cost_of_insured_vehicle,channel_min_Replacement_cost_of_insured_vehicle
0,00520d090d60ddb041b1c7043d77d5073bdcd464,2,2,2,2,1.000000,1.000000,1.0,1.000000,6526,...,6000000,200000,0,0.000000,0,0,753.7,107.671429,127.0,81.9
1,0090902477ffe105c4b342a37987a0b82103e7b0,15,360,388,382,1.077778,1.061111,24.0,25.866667,2459616,...,80300000,0,1185,0.662011,20,0,163924.8,91.578101,455.0,25.3


In [135]:
channel_attr['Insured_Amount1_per_area'] = channel_attr.channel_sum_Insured_Amount1 / channel_attr.channel_iply_area
channel_attr['Insured_Amount2_per_area'] = channel_attr.channel_sum_Insured_Amount2 / channel_attr.channel_iply_area
channel_attr['Insured_Amount3_per_area'] = channel_attr.channel_sum_Insured_Amount3 / channel_attr.channel_iply_area
channel_attr['Coverage_Deductible_if_applied_per_area'] = channel_attr.channel_sum_Coverage_Deductible_if_applied / channel_attr.channel_iply_area
channel_attr['Replacement_cost_of_insured_vehicle_per_area'] = channel_attr.channel_sum_Replacement_cost_of_insured_vehicle / channel_attr.channel_iply_area
channel_attr['Premium_per_area_per_area'] = channel_attr.channel_sum_Premium / channel_attr.channel_iply_area

In [136]:
channel_attr['Insured_Amount1_per_ply'] = channel_attr.channel_sum_Insured_Amount1 / channel_attr.channel_Policy_Number
channel_attr['Insured_Amount2_per_ply'] = channel_attr.channel_sum_Insured_Amount2 / channel_attr.channel_Policy_Number
channel_attr['Insured_Amount3_per_ply'] = channel_attr.channel_sum_Insured_Amount3 / channel_attr.channel_Policy_Number
channel_attr['Coverage_Deductible_if_applied_per_ply'] = channel_attr.channel_sum_Coverage_Deductible_if_applied / channel_attr.channel_Policy_Number
channel_attr['Replacement_cost_of_insured_vehicle_per_ply'] = channel_attr.channel_sum_Replacement_cost_of_insured_vehicle / channel_attr.channel_Policy_Number
channel_attr['Premium_per_ply_per_ply'] = channel_attr.channel_sum_Premium / channel_attr.channel_Policy_Number

In [137]:
channel_attr['Insured_Amount1_per_Customer'] = channel_attr.channel_sum_Insured_Amount1 / channel_attr["channel_Insured's_ID"]
channel_attr['Insured_Amount2_per_Customer'] = channel_attr.channel_sum_Insured_Amount2 / channel_attr["channel_Insured's_ID"]
channel_attr['Insured_Amount3_per_Customer'] = channel_attr.channel_sum_Insured_Amount3 / channel_attr["channel_Insured's_ID"]
channel_attr['Coverage_Deductible_if_applied_per_Customer'] = channel_attr.channel_sum_Coverage_Deductible_if_applied / channel_attr["channel_Insured's_ID"]
channel_attr['Replacement_cost_of_insured_vehicle_per_Customer'] = channel_attr.channel_sum_Replacement_cost_of_insured_vehicle / channel_attr["channel_Insured's_ID"]
channel_attr['Premium_per_Customer_per_Customer'] = channel_attr.channel_sum_Premium / channel_attr["channel_Insured's_ID"]

In [138]:
print(channel_attr.shape); channel_attr.head(2)

(875, 51)


,Distribution_Channel,channel_iply_area,channel_Insured's_ID,channel_Policy_Number,channel_Vehicle_identifier,avg_ply_per_customer,avg_car_per_customer,avg_customer_per_area,avg_ply_per_area,channel_sum_Premium,...,Insured_Amount3_per_ply,Coverage_Deductible_if_applied_per_ply,Replacement_cost_of_insured_vehicle_per_ply,Premium_per_ply_per_ply,Insured_Amount1_per_Customer,Insured_Amount2_per_Customer,Insured_Amount3_per_Customer,Coverage_Deductible_if_applied_per_Customer,Replacement_cost_of_insured_vehicle_per_Customer,Premium_per_Customer_per_Customer
0,00520d090d60ddb041b1c7043d77d5073bdcd464,2,2,2,2,1.000000,1.000000,1.0,1.000000,6526,...,1.085000e+07,0.000000,376.850000,3263.000000,3.100000e+06,1.500000e+06,1.085000e+07,0.000000,376.850000,3263.000000
1,0090902477ffe105c4b342a37987a0b82103e7b0,15,360,388,382,1.077778,1.061111,24.0,25.866667,2459616,...,3.231043e+07,3.054124,422.486598,6339.216495,3.250924e+06,2.424444e+06,3.482346e+07,3.291667,455.346667,6832.266667


In [139]:
channel_attr.to_feather('data/mdata/channel_attr')

### Final. 把所有資料merge成建模大表

In [140]:
ls data/mdata

all_data_v0807     area_attr             policy_sum_group2     X_trn_0904
all_data_v0807_v2  car                   policy_sum_group2_v2  X_trn_0906
all_data_v0825     car_v2                policy_sum_group3     X_trn_v0904
all_data_v0825_v2  channel_attr          policy_sum_group3_v2  X_tst_0827
all_data_v0831     clm                   trn_df_v0904          X_tst_0904
all_data_v0831_v2  man                   trn_final_v0904       X_tst_0906
all_data_v0905     policy_part1          tst_df_v0904          y_trn_0827
all_data_v0905_v2  policy_part1_v2       tst_final_v0904       y_trn_0904
all_data_v0906     policy_part1_v3       val_flg               y_trn_0906
all_data_v0906_v2  policy_sum_group1     xcol_keep_name.npy    y_trn_v0904
all_id_y           policy_sum_group1_v2  X_trn_0827


In [141]:
all_id_y = pd.read_feather('data/mdata/all_id_y')
man = pd.read_feather('data/mdata/man')
car = pd.read_feather('data/mdata/car_v2')
clm = pd.read_feather('data/mdata/clm')
policy_part1 = pd.read_feather('data/mdata/policy_part1_v2')
policy_sum_group1 = pd.read_feather('data/mdata/policy_sum_group1_v2')
policy_sum_group2 = pd.read_feather('data/mdata/policy_sum_group2_v2')
policy_sum_group3 = pd.read_feather('data/mdata/policy_sum_group3_v2')

# area_attr = pd.read_feather('data/mdata/area_attr')
channel_attr = pd.read_feather('data/mdata/channel_attr')

In [142]:
all_id_y = all_id_y.merge(man, on='Policy_Number')
all_id_y = all_id_y.merge(car, on='Policy_Number')
all_id_y = all_id_y.merge(policy_part1, on='Policy_Number')
all_id_y = all_id_y.merge(policy_sum_group1, on='Policy_Number')
all_id_y = all_id_y.merge(policy_sum_group2, on='Policy_Number')
all_id_y = all_id_y.merge(policy_sum_group3, on='Policy_Number')

all_id_y = all_id_y.merge(clm, how='left', on='Policy_Number')
all_id_y = all_id_y.merge(channel_attr, how='left', on='Distribution_Channel')

In [143]:
# v5. 拿掉area相關變數、新增channel相關變數
all_id_y.to_feather('data/mdata/all_data_v0906')

# v5. 新增area相關變數
# all_id_y.to_feather('data/mdata/all_data_v0905')

# v4. 去除90%以上為0的變數、新增多個彙總變數
# all_id_y.to_feather('data/mdata/all_data_v0831')

# v3. 增加clm，part1保留多類別變數
# all_id_y.to_feather('data/mdata/all_data_v0825')

# v2. 修改欄位名稱，part2資料的字尾保留_except_cancel
# all_id_y.to_feather('data/mdata/all_data_v0807')

# v1
# all_id_y.to_feather('data/mdata/all_data_v0802')

### Post-Processing

In [144]:
all_data = pd.read_feather('data/mdata/all_data_v0906')

In [145]:
print(len(all_data))
display_all(all_data.head(2))

351273


,Policy_Number,Next_Premium,Test_Flg,Insured's_ID,fassured,ibirth,dbirth,fsex,fmarriage,aassured_zip,iply_area,ins_year,ins_mon,age,di_equal,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,car_age,tax_1,tax_2,Cancellation,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,min_Insured_Amount1,max_Insured_Amount1,sum_Insured_Amount1,mean_Insured_Amount1,min_Insured_Amount2,max_Insured_Amount2,sum_Insured_Amount2,mean_Insured_Amount2,min_Insured_Amount3,max_Insured_Amount3,sum_Insured_Amount3,mean_Insured_Amount3,min_Coverage_Deductible_if_applied,max_Coverage_Deductible_if_applied,sum_Coverage_Deductible_if_applied,mean_Coverage_Deductible_if_applied,min_Premium,max_Premium,sum_Premium,mean_Premium,min_Premium_except_cancel,max_Premium_except_cancel,sum_Premium_except_cancel,mean_Premium_except_cancel,竊盜_max_Coverage_Deductible_if_applied,車損_max_Coverage_Deductible_if_applied,車責_max_Coverage_Deductible_if_applied,竊盜_mean_Coverage_Deductible_if_applied,車損_mean_Coverage_Deductible_if_applied,車責_mean_Coverage_Deductible_if_applied,竊盜_min_Coverage_Deductible_if_applied,車損_min_Coverage_Deductible_if_applied,車責_min_Coverage_Deductible_if_applied,竊盜_sum_Coverage_Deductible_if_applied,車損_sum_Coverage_Deductible_if_applied,車責_sum_Coverage_Deductible_if_applied,竊盜_max_Insured_Amount1,車損_max_Insured_Amount1,車責_max_Insured_Amount1,竊盜_mean_Insured_Amount1,車損_mean_Insured_Amount1,車責_mean_Insured_Amount1,竊盜_min_Insured_Amount1,車損_min_Insured_Amount1,車責_min_Insured_Amount1,竊盜_sum_Insured_Amount1,車損_sum_Insured_Amount1,車責_sum_Insured_Amount1,竊盜_max_Insured_Amount2,車損_max_Insured_Amount2,車責_max_Insured_Amount2,竊盜_mean_Insured_Amount2,車損_mean_Insured_Amount2,車責_mean_Insured_Amount2,竊盜_min_Insured_Amount2,車損_min_Insured_Amount2,車責_min_Insured_Amount2,竊盜_sum_Insured_Amount2,車損_sum_Insured_Amount2,車責_sum_Insured_Amount2,竊盜_max_Insured_Amount3,車損_max_Insured_Amount3,車責_max_Insured_Amount3,竊盜_mean_Insured_Amount3,車損_mean_Insured_Amount3,車責_mean_Insured_Amount3,竊盜_min_Insured_Amount3,車損_min_Insured_Amount3,車責_min_Insured_Amount3,竊盜_sum_Insured_Amount3,車損_sum_Insured_Amount3,車責_sum_Insured_Amount3,竊盜_max_Premium,車損_max_Premium,車責_max_Premium,竊盜_mean_Premium,車損_mean_Premium,車責_mean_Premium,竊盜_min_Premium,車損_min_Premium,車責_min_Premium,竊盜_sum_Premium,車損_sum_Premium,車責_sum_Premium,竊盜_max_Premium_except_cancel,車損_max_Premium_except_cancel,車責_max_Premium_except_cancel,竊盜_mean_Premium_except_cancel,車損_mean_Premium_except_cancel,車責_mean_Premium_except_cancel,竊盜_min_Premium_except_cancel,車損_min_Premium_except_cancel,車責_min_Premium_except_cancel,竊盜_sum_Premium_except_cancel,車損_sum_Premium_except_cancel,車責_sum_Premium_except_cancel,00I_Insured_Amount1,01A_Insured_Amount1,01J_Insured_Amount1,02K_Insured_Amount1,03L_Insured_Amount1,04M_Insured_Amount1,05E_Insured_Amount1,05N_Insured_Amount1,06F_Insured_Amount1,07P_Insured_Amount1,08H_Insured_Amount1,09@_Insured_Amount1,09I_Insured_Amount1,10A_Insured_Amount1,12L_Insured_Amount1,14E_Insured_Amount1,14N_Insured_Amount1,15F_Insured_Amount1,15O_Insured_Amount1,16G_Insured_Amount1,16P_Insured_Amount1,18@_Insured_Amount1,18I_Insured_Amount1,20B_Insured_Amount1,20K_Insured_Amount1,25G_Insured_Amount1,26H_Insured_Amount1,27I_Insured_Amount1,29B_Insured_Amount1,29K_Insured_Amount1,32N_Insured_Amount1,33F_Insured_Amount1,33O_Insured_Amount1,34P_Insured_Amount1,35H_Insured_Amount1,36I_Insured_Amount1,37J_Insured_Amount1,40M_Insured_Amount1,41E_Insured_Amount1,41N_Insured_Amount1,42F_Insured_Amount1,45@_Insured_Amount1,46A_Insured_Amount1,47B_Insured_Amount1,51O_Insured_Amount1,55J_Insured_Amount1,56B_Insured_Amount1,56K_Insured_Amount1,57C_Insured_Amount1,57L_Insured_Amount1,65K_Insured_Amount1,66C_Insured_Amount1,66L_Insured_Amount1,67D_Insured_Amount1,68E_Insured_Amount1,68N_Insured_Amount1,70G_Insured_A

#### 移除_except_cancel欄位的姊妹欄

In [146]:
# part1. 移除 _except_cancel
all_cols = all_data.columns.values

drop_cols = []
for col in all_cols:
    if '_except_cancel' in col:
        drop_cols.append(col.replace('_except_cancel', ''))

# drop_cols

In [147]:
all_data.drop(columns=drop_cols, inplace=True)

#### 把all_data再重新掛上險種分類

In [148]:
# flt = (all_data['竊盜_Premium_except_cancel'] > 0)
flt = (all_data['竊盜_sum_Premium_except_cancel'] > 0)
all_data['竊盜_flg'] = 0
all_data.loc[flt, '竊盜_flg'] = 1

In [149]:
flt = (all_data['車損_sum_Premium_except_cancel'] > 0)
all_data['車損_flg'] = 0
all_data.loc[flt, '車損_flg'] = 1

In [150]:
flt = (all_data['車責_sum_Premium_except_cancel'] > 0)
all_data['車責_flg'] = 0
all_data.loc[flt, '車責_flg'] = 1

#### 增加以Next_Premium做為分類的Flg

In [151]:
all_data.head()

,Policy_Number,Next_Premium,Test_Flg,Insured's_ID,fassured,ibirth,dbirth,fsex,fmarriage,aassured_zip,...,Premium_per_ply_per_ply,Insured_Amount1_per_Customer,Insured_Amount2_per_Customer,Insured_Amount3_per_Customer,Coverage_Deductible_if_applied_per_Customer,Replacement_cost_of_insured_vehicle_per_Customer,Premium_per_Customer_per_Customer,竊盜_flg,車損_flg,車責_flg
0,8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414,0,Pf463063087505010b011330565193eae3213beca,1,11/1980,11/1980,1,1,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,...,5507.558598,2.854128e+06,1.683382e+06,3.496723e+07,135.422373,467.921721,5870.499734,0,0,1
1,3f9f289dd1b20494a2fe4b1f0fe2d78333c97eda,8236,0,Sd873f723d823d9109c82aa754ff26ae916822c5d,1,11/1956,03/1982,2,2,62362fbf882221470f099d8d0a119df93ce19831,...,5507.558598,2.854128e+06,1.683382e+06,3.496723e+07,135.422373,467.921721,5870.499734,1,1,1
2,9864c955e499d1fa908f5dc7292054a232f01722,6153,0,J4f8382fcbf2bbc4fbe722a939cf8aa8bcf3c8fa5,1,03/1954,03/1954,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,...,5507.558598,2.854128e+06,1.683382e+06,3.496723e+07,135.422373,467.921721,5870.499734,1,1,1
3,92170afa649706baf10774dc0a46e67617c0d50d,12135,0,Hbea126c5bc9c81d1cf4de28b2f1772ec195d9a24,1,09/1978,09/1978,1,1,01592d51db5afd0165cb73baca5c0b340c4889f1,...,5507.558598,2.854128e+06,1.683382e+06,3.496723e+07,135.422373,467.921721,5870.499734,1,1,1
4,d16c11ef69f5f6bac02ed18fb1e372394563697f,4345,0,T79292b97583a9c2e3f818f8745043ea5ce4a6aee,1,10/1960,10/1960,1,1,310b86e0b62b828562fc91c7be5380a992b2786a,...,5507.558598,2.854128e+06,1.683382e+06,3.496723e+07,135.422373,467.921721,5870.499734,0,1,0


In [152]:
all_data['Classifer_flg'] = 'others'

In [153]:
trn_df = all_data[all_data.Test_Flg == '0']
tst_df = all_data[all_data.Test_Flg == '1']

In [154]:
# value_list = trn_df.Next_Premium.describe(percentiles=[0.2, 0.9, 0.99])

# value_list = pd.DataFrame(value_list).reset_index()
# value_list.columns = ['type', 'threshold']
# value_list

In [155]:
# thres_99 = int(value_list[value_list.type == '99%'].threshold)
# thres_90 = int(value_list[value_list.type == '90%'].threshold)

In [156]:
flt_0 = (trn_df.Next_Premium == 0)
# flt_1 = (thres_99 >= trn_df.Next_Premium) & (trn_df.Next_Premium >= thres_90)
# flt_2 = (trn_df.Next_Premium > thres_99)

In [157]:
trn_df.loc[flt_0, 'Classifer_flg'] = '續約金額為零'
# trn_df.loc[flt_1, 'Classifer_flg'] = '續約金額在前90~99%'
# trn_df.loc[flt_2, 'Classifer_flg'] = '續約金額在前99%以上'

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [158]:
# tst_df.loc[:, 'Classifer_flg'] = '未定義'

In [159]:
final_df = pd.concat([trn_df, tst_df])

In [160]:
final_df.dtypes

Policy_Number                                        object
Next_Premium                                          int64
Test_Flg                                             object
Insured's_ID                                         object
fassured                                              int64
ibirth                                               object
dbirth                                               object
fsex                                                 object
fmarriage                                            object
aassured_zip                                         object
iply_area                                            object
ins_year                                              int64
ins_mon                                               int64
age                                                   int64
di_equal                                              int64
Vehicle_identifier                                   object
Vehicle_Make_and_Model1                 

#### export to feather

In [161]:
final_df.to_feather('data/mdata/all_data_v0906_v2')